# Mohammed Suhail
# 94143161

In [2]:
from collections import Counter
from gensim.models import Word2Vec
from random import random
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from torch import nn
from torch.autograd import Variable

import numpy as np
import torch
import torch.nn.functional as F

import torch.utils.data.sampler as sampler

import time
import math

# Data Acquisition

For this assignment, you must download the data and extract it into `data/`. The dataset contains two files, both containing a single caption on each line. We should have 415,795 sentences in the training captions and 500 sentences in the validation captions.

To download the data, run the following directly on your server: `wget https://s3-us-west-2.amazonaws.com/cpsc532l-data/a3_data.zip`

In [3]:
# Load the data into memory.
train_sentences = [line.strip() for line in open("data/mscoco_train_captions.txt").readlines()]
val_sentences = [line.strip() for line in open("data/mscoco_val_captions.txt").readlines()]

print(len(train_sentences))
print(len(val_sentences))
print(train_sentences[0])

415795
500
A very clean and well decorated empty bathroom


# Preprocessing

The code provided below creates word embeddings for you to use. After creating the vocabulary, we construct both one-hot embeddings and word2vec embeddings. 

All of the packages utilized should be installed on your Azure servers, however you will have to download an NLTK corpus. To do this, follow the instructions below:

1. SSH to your Azure server
2. Open up Python interpreter
3. `import nltk`
4. `nltk.download()`

    You should now see something that looks like:

    ```
    >>> nltk.download()
    NLTK Downloader
    ---------------------------------------------------------------------------
        d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
    ---------------------------------------------------------------------------
    Downloader> 

    ```

5. `d punkt`
6. Provided the download finished successfully, you may now exit out of the Python interpreter and close the SSH connection.

Please look through the functions provided below **carefully**, as you will need to use all of them at some point in your assignment.

In [ ]:
sentences = train_sentences

# Lower-case the sentence, tokenize them and add <SOS> and <EOS> tokens
sentences = [["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] for sentence in sentences]

# Create the vocabulary. Note that we add an <UNK> token to represent words not in our vocabulary.
vocabularySize = 1000 #1001
word_counts = Counter([word for sentence in sentences for word in sentence])
vocabulary = ["<UNK>"] + [e[0] for e in word_counts.most_common(vocabularySize-1)] #-2 + [<PAD>]
word2index = {word:index for index,word in enumerate(vocabulary)}
one_hot_embeddings = np.eye(vocabularySize)

# Build the word2vec embeddings
wordEncodingSize = 300
filtered_sentences = [[word for word in sentence if word in word2index] for sentence in sentences]
w2v = Word2Vec(filtered_sentences, min_count=0, size=wordEncodingSize)
w2v_embeddings = np.concatenate((np.zeros((1, wordEncodingSize)), w2v.wv.syn0))

In [15]:
# Define the max sequence length to be the longest sentence in the training data. 
maxSequenceLength = max([len(sentence) for sentence in sentences])

def preprocess_numberize(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into list of numbers (denoting the index into the vocabulary).
    """
    tokenized = word_tokenize(sentence.lower())
        
    # Add the <SOS>/<EOS> tokens and numberize (all unknown words are represented as <UNK>).
    tokenized = ["<SOS>"] + tokenized + ["<EOS>"]
    numberized = [word2index.get(word, 0) for word in tokenized]
    
    return numberized

def preprocess_one_hot(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into a numpy array of one-hot vectors.
    """
    numberized = preprocess_numberize(sentence)
    
    # Represent each word as it's one-hot embedding
    one_hot_embedded = one_hot_embeddings[numberized]
    
    return one_hot_embedded

def preprocess_word2vec(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into a numpy array of word2vec embeddings.
    """
    numberized = preprocess_numberize(sentence)
    
    # Represent each word as it's one-hot embedding
    w2v_embedded = w2v_embeddings[numberized]
    
    return w2v_embedded

def compute_bleu(reference_sentence, predicted_tokenized):
    """
    Given a reference sentence, and a predicted sentence, compute the BLEU similary between them.
    """
    reference_tokenized = word_tokenize(reference_sentence.lower())
    #predicted_tokenized = word_tokenize(predicted_sentence.lower())
    return sentence_bleu([reference_tokenized], predicted_tokenized)

# 1. Building a Language Decoder

We now implement a language decoder. For now, we will have the decoder take a single training sample at a time (as opposed to batching). For our purposes, we will also avoid defining the embeddings as part of the model and instead pass in embedded inputs. While this is sometimes useful, as it learns/tunes the embeddings, we avoid doing it for the sake of simplicity and speed.

Remember to use LSTM hidden units!

In [5]:
use_cuda = False

In [6]:
class DecoderLSTM(nn.Module):
    # Your code goes here
    def __init__(self,input_size,hidden_size,output_size):
        
        super(DecoderLSTM,self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim = 1)
    
    def forward(self, input, hidden,memory):
        
        output, (hidden,memory) = self.lstm(input,(hidden,memory))
        output = self.out(output)
        return output,(hidden,memory)
    
    def initHiddenandMem(self):
        
        result1 = Variable(torch.zeros(1,1,self.hidden_size))
        result2 = Variable(torch.zeros(1,1,self.hidden_size))
        #result = result.type(torch.LongTensor)
        if use_cuda:
            return result1.cuda(),result2.cuda()
        else:
            return result1,result2
        
        
   

In [7]:
temp = word2index.get("<EOS>", 0)     
EOS_one_hot = one_hot_embeddings[temp]
print("the index for EOS is ",temp)
EOS_idx = temp
word2index.get("<EOS>",0)

the index for EOS is  3


3

In [40]:
test_input = preprocess_one_hot(train_sentences[0])
print(test_input)
idx = np.where(test_input[0] == 1)
print(idx[0][0])

[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
2


# 2. Training a Language Decoder

We must now train the language decoder we implemented above. An important thing to pay attention to is the [inputs for an LSTM](http://pytorch.org/docs/master/nn.html#torch.nn.LSTM).

In [35]:
def train(target_variable, 
          decoder, 
          decoder_optimizer, 
          criterion, 
          embeddings=one_hot_embeddings): 
    """
    Given a single training sample, go through a single step of training.
    """
    raw_sentence = target_variable
    raw_sentence = word_tokenize(raw_sentence.lower()) + ["<EOS>"]
    
    #print(raw_sentence)
    #target_variable = preprocess_one_hot(target_variable)
    target_variable = preprocess_word2vec(target_variable)
    
    target_variable = target_variable[1:,:]
    decoder_optimizer.zero_grad()
    
    #print(target_variable[0,:].shape)
    #decoder_input = Variable(torch.from_numpy((target_variable[0,:]))).view(-1,1,vocabularySize)
    
    decoder_hidden,decoder_memory = decoder.initHiddenandMem()
    
    target_length = target_variable.shape[0]
    
    decoder_input = Variable(torch.FloatTensor(target_variable[0,:]).view(-1,1,decoder.input_size))
    
    loss = 0
    for i in range(target_length-1):
        #print(i)
        
        
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
        decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input,decoder_hidden,decoder_memory)
        
        topv, topi = decoder_output.data.topk(1)
            
        
        #print(topi.cpu())
        ni = topi[0][0]
        #print(ni)
        
        decoder_input = Variable(torch.FloatTensor(embeddings[ni[0]])).view(1,1,decoder.input_size)
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
        '''
        for j in range(vocabularySize):
                if(target_variable[i][j] == 1):
                    idx = j
        '''
        
        #print(raw_sentence[i+1])
        idx = word2index.get(raw_sentence[i+1],0)
        
        #target = Variable(torch.LongTensor([idx])).cuda()
        target = Variable(torch.LongTensor([idx]))

        prediction = decoder_output.view(1,vocabularySize)
        
        loss += criterion(prediction,target)
    
        if ni[0] == EOS_idx:
            break;
    
    loss.backward()
    decoder_optimizer.step()
        
    return loss.data[0]/target_length
    
    

In [302]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
# Train the model and monitor the loss. Remember to use Adam optimizer and CrossEntropyLoss
plot_losses = []
print_loss_total = 0  # Reset every print_every
plot_loss_total = 0

def trainIters(decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    for iter in range(1,n_iters + 1):
        
        target_variable = train_sentences[iter]
        if len(target_variable.strip()) == 0:
            continue
        
        loss = train(target_variable,decoder,decoder_optimizer,criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            
            print_loss_avg = print_loss_total/print_every
            print_loss_total = 0
            
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
        

        
 

In [301]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker



def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)    

    

In [72]:
len(train_sentences)

415795

In [20]:
lr = 10**(-4)
hidden_size = 300
output_size = vocabularySize

decoder = DecoderLSTM(hidden_size,output_size).cuda() if use_cuda else DecoderLSTM(hidden_size,output_size)

#trainIters(decoder,415795,1000,100,lr)   
trainIters(decoder,100,10,10,lr) 

0m 0s (- 0m 7s) (10 10%) 6.2998
0m 1s (- 0m 7s) (20 20%) 6.3575
0m 3s (- 0m 7s) (30 30%) 6.2995
0m 4s (- 0m 6s) (40 40%) 5.6515
0m 5s (- 0m 5s) (50 50%) 6.2954
0m 6s (- 0m 4s) (60 60%) 5.5591
0m 6s (- 0m 2s) (70 70%) 4.2185
0m 7s (- 0m 1s) (80 80%) 3.2653
0m 7s (- 0m 0s) (90 90%) 3.6731
0m 8s (- 0m 0s) (100 100%) 4.0414


0m 34s (- 235m 24s) (1000 0%) 3.6317
1m 16s (- 264m 4s) (2000 0%) 3.8989
1m 59s (- 273m 1s) (3000 0%) 3.7776
2m 39s (- 273m 36s) (4000 0%) 3.5679
3m 18s (- 272m 28s) (5000 1%) 3.4252
3m 58s (- 271m 51s) (6000 1%) 3.4387
5m 56s (- 346m 56s) (7000 1%) 3.5425
6m 40s (- 340m 0s) (8000 1%) 3.5807
7m 23s (- 334m 5s) (9000 2%) 3.4861
8m 7s (- 329m 40s) (10000 2%) 3.6070
8m 52s (- 326m 52s) (11000 2%) 3.5872
9m 38s (- 324m 11s) (12000 2%) 3.6073
10m 23s (- 321m 46s) (13000 3%) 3.5506
11m 7s (- 319m 25s) (14000 3%) 3.5752
11m 50s (- 316m 33s) (15000 3%) 3.4986
12m 33s (- 313m 55s) (16000 3%) 3.4319
13m 17s (- 311m 46s) (17000 4%) 3.5562
14m 0s (- 309m 44s) (18000 4%) 3.5219
14m 43s (- 307m 29s) (19000 4%) 3.4150
15m 25s (- 305m 19s) (20000 4%) 3.3716
16m 5s (- 302m 30s) (21000 5%) 3.2187
16m 47s (- 300m 33s) (22000 5%) 3.3760
17m 29s (- 298m 35s) (23000 5%) 3.4121
18m 10s (- 296m 38s) (24000 5%) 3.2847
18m 49s (- 294m 22s) (25000 6%) 3.2479
19m 31s (- 292m 40s) (26000 6%) 3.3338
20m 15s (- 291m

KeyboardInterrupt: 

In [107]:
torch.save(decoder, 'Decoder_upto_30000.pt')

/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type DecoderLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
test_decoder = torch.load('Decoder_upto_30000.pt')

In [117]:
torch.save(decoder,'Decoder_with_eos_50000.pt')

/anaconda/envs/py35/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type DecoderLSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [68]:
learning_rate = 3*10**(-4)

hidden_size = 300
output_size = vocabularySize

decoder = DecoderLSTM(hidden_size,output_size)

decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss()

test_input = train_sentences[0]
#print(test_input)
train(test_input,decoder,decoder_optimizer,criterion)

A very clean and well decorated empty bathroom


6.1358591715494795

In [50]:
decoder

DecoderLSTM(
  (lstm): LSTM(1000, 300)
  (out): Linear(in_features=300, out_features=1000)
)

# 3. Building Language Decoder MAP Inference

We now define a method to perform inference with our decoder and test it with a few different starting words. This code will be fairly similar to your training function from part 2.

In [122]:
def inference(decoder, init_word, embeddings=one_hot_embeddings, max_length=maxSequenceLength):
    # Your code goes here
    
    word_idx = word2index.get(init_word,0)
    init_word_vector = embeddings[word_idx]
    
    
    decoder_input = Variable(torch.FloatTensor(init_word_vector)).view(1,1,vocabularySize)
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
    decoder_hidden,decoder_memory = decoder.initHiddenandMem()
    decoded_words = []
    
    for i in range(max_length):
        
        decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input,decoder_hidden,decoder_memory)
        
        topv, topi = decoder_output.data.topk(1)
        
        ni = topi[0][0]
        
        decoder_input = Variable(torch.FloatTensor(embeddings[ni[0]])).view(1,1,vocabularySize)
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
        if ni[0] == EOS_idx:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(vocabulary[ni[0]])
        
    return decoded_words
            
        

temp = inference(test_decoder, init_word="the")
print(temp)
print(inference(test_decoder, init_word="man"))
print(inference(test_decoder, init_word="woman"))
print(inference(test_decoder, init_word="dog"))

['the', 'man', 'is', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<EOS>']
['man', 'with', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<EOS>']
['woman', 'a', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<EOS>']
['dog', 'of', 'a', 'a', 'a', 'a', 'a', 'a', '.', '<EOS>']


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:16: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().


# 4. Building Language Decoder Sampling Inference

We must now modify the method defined in part 3, to sample from the distribution outputted by the LSTM rather than taking the most probable word.

It might be useful to take a look at the output of your model and (depending on your implementation) modify it so that the outputs sum to 1. 

In [204]:
maxSequenceLength = 10
def sampling_inference(decoder, init_word, embeddings=one_hot_embeddings, max_length=maxSequenceLength):
    # Your code goes here
    
    
    
    decoded_words_all = []
    
    softmax = nn.Softmax()
    
    for j in range(5):
        
        decoded_words = []
        word_idx = word2index.get(init_word,0)
        init_word_vector = embeddings[word_idx]


        decoder_input = Variable(torch.FloatTensor(init_word_vector)).view(1,1,vocabularySize)
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

        decoder_hidden,decoder_memory = decoder.initHiddenandMem()

        for i in range(max_length):

            decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input,decoder_hidden,decoder_memory)

            #topv, topi = decoder_output.data.topk(1)
            decoder_output = decoder_output.view(1,vocabularySize)
            output = w(decoder_output).data.cpu()

            output = output.numpy()
            output = output.tolist()

            out_iter = sampler.WeightedRandomSampler(output,len(output))

            for idx in out_iter:
                ni = idx
                break

            #print(ni)    
            decoder_input = Variable(torch.FloatTensor(embeddings[ni[0]])).view(1,1,vocabularySize)
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            if ni[0] == EOS_idx:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(vocabulary[ni[0]])
                
        decoded_words_all.append(decoded_words)
    #print(len(decoded_words_all))
    return decoded_words_all

# Print the results with sampling_inference by drawing 5 samples per initial word, requiring to run 
# the code below 5 times
print(sampling_inference(decoder, init_word="the"))
print("\n\n\n")
print(sampling_inference(decoder, init_word="man"))
print("\n\n\n")
print(sampling_inference(decoder, init_word="woman"))
print("\n\n\n")
print(sampling_inference(decoder, init_word="dog"))

[['photos', 'fruits', '.', 'and', 'close', 'a', 'of', 'small', 'small', 'near', "'s", 'a', 'at', 'standing', 'on', 'with', 'toilet', 'of', 'in', 'traveling', 'white', 'kitchen', 'parked', 'bowl', 'window', 'next', 'in', 'motorcycles', '<UNK>', 'monitor', 'to', 'motorcycle', 'motorcycle', 'small', 'woman', 'public', 'bike', 'wearing', 'stop', 'different', 'flying', 'driving', 'on', 'looking', 'bathroom', 'a', 'a', 'motorcycle', 'motorcycle', 'an', 'the', 'and', 'near', 'next', '<EOS>'], ['the', 'old', 'airplane', 'and', 'next', 'are', 'in', 'looking', 'is', 'sitting', 'the', 'standing', 'a', 'no', 'on', 'bathroom', 'cars', 'airplane', 'an', 'an', '<UNK>', 'vase', 'on', 'with', 'on', 'to', 'parked', 'on', 'flying', ',', 'of', 'track', '.', '<EOS>'], ['the', 'row', 'bowl', 'a', 'of', 'in', 'has', '<UNK>', '<UNK>', 'dog', 'motorcycle', 'on', 'with', 'old', 'motorcycle', '<UNK>', 'and', 'tall', 'with', 'lights', 'in', 'room', 'wooden', 'cat', 'toilets', 'image', 'the', 'through', 'near', 'o

In [193]:
init_word="the"
word_idx = word2index.get(init_word,0)
init_word_vector = one_hot_embeddings[word_idx]
    
    
decoder_input = Variable(torch.FloatTensor(init_word_vector)).view(1,1,vocabularySize)
decoder_input = decoder_input.cuda() if use_cuda else decoder_input

decoder_hidden,decoder_memory = decoder.initHiddenandMem()
decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input,decoder_hidden,decoder_memory)


softmax = nn.Softmax()
decoder_output = decoder_output.view(1,vocabularySize)
output = softmax(decoder_output).data.cpu()

output = output.numpy()
output = output.tolist()

print(type(output))
print(len(output))
#print(output)
out_iter = sampler.WeightedRandomSampler(output,len(output))
for i in out_iter:
    idx = i
    break
print(i)
temp = one_hot_embeddings[i[0]]

<class 'list'>
1

 725
[torch.LongTensor of size 1]



# 5.  Building Language Encoder

We now build a language encoder, which will encode an input word by word, and ultimately output a hidden state that we can then be used by our decoder.

In [9]:
class EncoderLSTM(nn.Module):
    # Your code goes here
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)

    def forward(self, input, hidden,memory):
        
        output, (hidden,memory) = self.lstm(input,(hidden,memory))
        return output,(hidden,memory)

    def initHiddenandMem(self):
        
        result1 = Variable(torch.zeros(1,1,self.hidden_size))
        result2 = Variable(torch.zeros(1,1,self.hidden_size))
        #result = result.type(torch.LongTensor)
        if use_cuda:
            return result1.cuda(),result2.cuda()
        else:
            return result1,result2
        
# Initialize the encoder with a hidden size of 300.

hidden_size =300

encoder = EncoderLSTM(vocabularySize,hidden_size).cuda() if use_cuda else EncoderLSTM(vocabularySize,hidden_size) 

# 6. Connecting Encoder to Decoder and Training End-to-End

We now connect our newly created encoder with our decoder, to train an end-to-end seq2seq architecture. 

It's likely that you'll be able to re-use most of your code from part 2. For our purposes, the only interaction between the encoder and the decoder is that the *last hidden state of the encoder is used as the initial hidden state of the decoder*. 

In [19]:
# Your code goes here
MAX_LENGTH = 30
def train_enc_dec(target_variable, encoder,decoder,
                  encoder_optimizer, decoder_optimizer, criterion, 
                  embeddings=one_hot_embeddings,max_length = MAX_LENGTH): 
    """
    Given a single training sample, go through a single step of training.
    """
    raw_sentence = target_variable
    raw_sentence = word_tokenize(raw_sentence.lower()) + ["<EOS>"]
    
    target_variable = preprocess_one_hot(target_variable)
    
    input_variable = target_variable[1:,:]
    input_length = input_variable.shape[0]
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    #encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    #encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    encoder_hidden, encoder_memory = encoder.initHiddenandMem()

    for ei in range(input_length):
        
        encoder_input = Variable(torch.FloatTensor(input_variable[ei,:]).view(-1,1,vocabularySize))
        encoder_input = encoder_input.cuda() if use_cuda else encoder_input
        
        encoder_output, (encoder_hidden,encoder_memory) = encoder(encoder_input, 
                                                                  encoder_hidden,encoder_memory)
        #encoder_outputs[ei] = encoder_output[0][0]
    
    #decoder_hidden,decoder_memory = decoder.initHiddenandMem()
    decoder_hidden,decoder_memory = encoder_hidden,encoder_memory
    
    #print(decoder_input.shape)
    #print(decoder_hidden.shape)
    #print(decoder_memory.shape)
    
    target_length = target_variable.shape[0]
    
    
    loss = 0
    for i in range(target_length):
        #print(i)
        
        decoder_input = Variable(torch.FloatTensor(target_variable[0,:]).view(-1,1,vocabularySize))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
        decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input,decoder_hidden,decoder_memory)
        
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]        
        
        idx = word2index.get(raw_sentence[i+1],0)
        
        target = Variable(torch.LongTensor([idx])).cuda() if use_cuda else Variable(torch.LongTensor([idx]))
        prediction = decoder_output.view(1,vocabularySize)
        loss += criterion(prediction,target)
    
        if ni[0] == EOS_idx:
            break;
    
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0]/target_length

In [8]:
# Train the model and monitor the loss. Remember to use Adam optimizer and CrossEntropyLoss
plot_losses = []
print_loss_total = 0  # Reset every print_every
plot_loss_total = 0

def trainIters(encoder,decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    encoder_optimizer = torch.optim.Adam(encoder.parameters(),lr = learning_rate)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    for iter in range(1,n_iters + 1):
        
        target_variable = train_sentences[iter]
        
        if len(target_variable.strip()) == 0:
            continue
        
        loss = train_enc_dec(target_variable, encoder,decoder,encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            
            print_loss_avg = print_loss_total/print_every
            print_loss_total = 0
            
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            

In [18]:
learning_rate = 5*10**(-4)

hidden_size = 300
input_size = vocabularySize
output_size = vocabularySize

encoder = EncoderLSTM(input_size,hidden_size).cuda() if use_cuda else EncoderLSTM(input_size,hidden_size)
decoder = DecoderLSTM(hidden_size,output_size).cuda() if use_cuda else DecoderLSTM(hidden_size,output_size)

trainIters(encoder,decoder, 5000, 100 , 100, learning_rate)

0m 12s (- 9m 48s) (100 2%) 4.9550
0m 24s (- 9m 56s) (200 4%) 4.5600
0m 36s (- 9m 35s) (300 6%) 3.8298
0m 49s (- 9m 24s) (400 8%) 3.8121
1m 1s (- 9m 9s) (500 10%) 3.7235
1m 12s (- 8m 53s) (600 12%) 3.5164
1m 25s (- 8m 42s) (700 14%) 3.6821
1m 37s (- 8m 29s) (800 16%) 3.4479
1m 49s (- 8m 18s) (900 18%) 3.2932
2m 0s (- 8m 1s) (1000 20%) 3.1627
2m 13s (- 7m 53s) (1100 22%) 3.4244
2m 27s (- 7m 45s) (1200 24%) 3.2760
2m 42s (- 7m 43s) (1300 26%) 3.3127
2m 56s (- 7m 33s) (1400 28%) 3.2369
3m 8s (- 7m 19s) (1500 30%) 3.2101
3m 21s (- 7m 7s) (1600 32%) 3.4570
3m 34s (- 6m 55s) (1700 34%) 3.5305
3m 47s (- 6m 43s) (1800 36%) 3.5979
3m 59s (- 6m 30s) (1900 38%) 3.2446
4m 12s (- 6m 18s) (2000 40%) 3.3308
4m 24s (- 6m 5s) (2100 42%) 3.2114
4m 37s (- 5m 52s) (2200 44%) 3.1870
4m 50s (- 5m 40s) (2300 46%) 3.3592
5m 3s (- 5m 28s) (2400 48%) 3.3306
5m 16s (- 5m 16s) (2500 50%) 3.4357
5m 29s (- 5m 3s) (2600 52%) 3.3733
5m 42s (- 4m 51s) (2700 54%) 3.4902
5m 55s (- 4m 39s) (2800 56%) 3.4277
6m 8s (- 4m 26

In [60]:
learning_rate = 3*10**(-4)

hidden_size = 300
input_size = vocabularySize
output_size = vocabularySize

encoder = EncoderLSTM(input_size,hidden_size).cuda() if use_cuda else EncoderLSTM(input_size,hidden_size)
decoder = DecoderLSTM(hidden_size,output_size).cuda() if use_cuda else DecoderLSTM(hidden_size,output_size)

encoder_optimizer = torch.optim.Adam(encoder.parameters(),lr = learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss()

train_enc_dec(test_input, encoder,decoder,encoder_optimizer, decoder_optimizer, criterion)

6.917446136474609

# 7. Testing 

We must now define a method that allows us to do inference using the seq2seq architecture. We then run the 500 validation captions through this method, and ultimately compare the **reference** and **generated** sentences using our **BLEU** similarity score method defined above, to identify the average BLEU score.

In [13]:
def seq2seq_inference(sentence, embeddings=one_hot_embeddings, max_length=maxSequenceLength):
    
    sentence = preprocess_one_hot(sentence)
    #print(sentence.shape)
    input_variable = sentence
    #print(type(input_variable))
    input_length = input_variable.shape[0]
    
    word_idx = word2index.get("<SOS>",0)
    init_word_vector = embeddings[word_idx]
    
    encoder_hidden,encoder_memory = encoder.initHiddenandMem()
    
    for ei in range(1,input_length):
        
        encoder_input = Variable(torch.FloatTensor(input_variable[ei,:]).view(-1,1,vocabularySize))
        encoder_input = encoder_input.cuda() if use_cuda else encoder_input
        
        encoder_output, (encoder_hidden,encoder_memory) = encoder(encoder_input, 
                                                                  encoder_hidden,encoder_memory)
    
    
    decoder_input = Variable(torch.FloatTensor(init_word_vector)).view(1,1,vocabularySize)
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
    decoder_hidden,decoder_memory = encoder_hidden,encoder_memory
    decoded_words = []
    
    for i in range(max_length):
        
        decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input,decoder_hidden,decoder_memory)
        
        topv, topi = decoder_output.data.topk(1)
        
        ni = topi[0][0]
        
        decoder_input = Variable(torch.FloatTensor(embeddings[ni[0]])).view(1,1,vocabularySize)
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
        if ni[0] == EOS_idx:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(vocabulary[ni[0]])
        
    return decoded_words

In [23]:
# Perform inference for all validation sequences and report the average BLEU score
    # Your code goes here
test_sentence = val_sentences[1]
print(seq2seq_inference(test_sentence, embeddings=one_hot_embeddings, max_length=maxSequenceLength))

NameError: name 'init_word_vector' is not defined

# 8. Encoding as Generic Feature Representation

We now use the final hidden state of our encoder, to identify the nearest neighbor amongst the training sentences for each sentence in our validation data.

It would be effective to first define a method that would generate all of the hidden states and store these hidden states **on the CPU**, and then loop over the generated hidden states to identify/output the nearest neighbors.

In [42]:
def final_encoder_hidden(sentence):
    # Your code goes here
    embeddings = one_hot_embeddings
    
    encoder_hidden,encoder_memory = encoder.initHiddenandMem()
    
    for word in sentence[1:]:
        
        word_idx = word2index.get(word,0)
        word_vector = embeddings[word_idx]
        
        encoder_input = Variable(torch.FloatTensor(word_vector).view(-1,1,vocabularySize))
        encoder_input = encoder_input.cuda() if use_cuda else encoder_input
        
        encoder_output, (encoder_hidden,encoder_memory) = encoder(encoder_input, 
                                                                  encoder_hidden,encoder_memory)
        
        
    return encoder_hidden.view(1,hidden_size).data.numpy()
    

# Now run all training data and validation data to store hidden states
    # Your code goes here

In [ ]:
val_sentences_tokenized = [word_tokenize(sentence.lower()) + ["<EOS>"] for sentence in val_sentences]

In [50]:
val_features = []
for sentence in val_sentences_tokenized:
    
    feature = final_encoder_hidden(sentence)
    val_features.append(feature)
    

In [ ]:
train_features = []
for sentence in sentences:
    
    feature = final_encoder_hidden(sentence)
    train_features.append(feature)
    
    

In [47]:
# Now get nearest neighbors and print


# 9. Effectiveness of word2vec

We now repeat everything done above using word2vec embeddings in place of one-hot embeddings. This will require re-running steps 1-8.

In [36]:
input_size = 300
hidden_size = 300
output_size = vocabularySize

decoder_w2v = DecoderLSTM(input_size,hidden_size,output_size)

learning_rate = 3*10**(-4)
decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss()

test_input = train_sentences[0]

train(test_input, decoder_w2v, decoder_optimizer, criterion,w2v_embeddings)

6.1472727457682295

# BONUS : Training LSTM Using Mini-batches 

In [372]:
# Your code goes hereb
def train_enc_dec_batch(batch_size,target_variable, encoder,decoder,
                  encoder_optimizer, decoder_optimizer, criterion, 
                  embeddings=one_hot_embeddings,transform = preprocess_one_hot): 
    """
    Given a single training sample, go through a single step of training.
    """
    #sentence = target_variable
    
    raw_sentences = target_variable
    raw_sentences = [["<SOS>"]+ word_tokenize(raw_sentence.lower()) + ["<EOS>"] for raw_sentence in raw_sentences]
    raw_sentences = sorted(raw_sentences,key = len,reverse = True)
    max_sentence_length = len(raw_sentences[0])
    pad_sentences = []
    for x in raw_sentences:
        
        temp = x + ["<PAD>"]*(max_sentence_length - len(x)) if max_sentence_length != len(x) else x
        pad_sentences.append(temp)
    
    #print(pad_sentences)
    raw_sentences = pad_sentences
    
    
    #print(raw_sentences)
    
    target_variable = [transform(sentence) for sentence in target_variable]
    target_variable = sorted(target_variable,key=len, reverse=True)
    
    lengths = [len(x) for x in target_variable]
    #print(lengths)
    
    max_length = len(target_variable[0]) 
    

    input_list = [target_variable[k][1:,:] for k in range(batch_size)]
    
        
    input_lengths = [length - 1 for length in lengths]

    input_variable = torch.zeros(max_length -1,batch_size,vocabularySize) # -1 for removing sos
    decoder_input_variable = torch.zeros(1,batch_size,vocabularySize)
    
    for i in range(batch_size):
        input_variable[:len(input_list[i]),i,:] = torch.from_numpy(np.array(input_list[i]))
        decoder_input_variable[0,i,:] = torch.from_numpy(np.array(target_variable[i][0,:]))
        
    
    input_variable = Variable(input_variable)
    input_variable = input_variable.cuda() if use_cuda else input_variable
    
    decoder_input_variable = Variable(decoder_input_variable)
    decoder_input_variable = decoder_input_variable.cuda() if use_cuda else decoder_input_variable
    
    
    input_variable = torch.nn.utils.rnn.pack_padded_sequence(input_variable,input_lengths)
   
    print(input_variable.data.shape)
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    encoder_hidden, encoder_memory = encoder.initHiddenandMem()
    
    encoder_ouptput, (encoder_hidden,encoder_memory) = encoder(input_variable,
                                                               encoder_hidden,encoder_memory)
    
    
    decoder_hidden,decoder_memory = encoder_hidden,encoder_memory
    
    

    loss = 0
    for i in range(max_length - 1):
        #print(i)
        
        decoder_output,(decoder_hidden,decoder_memory) = decoder(decoder_input_variable,decoder_hidden,decoder_memory)
        
        topv, topi = decoder_output.data.topk(1)
        
        ni = topi[0]
        
        decoder_input_variable = Variable(torch.FloatTensor(embeddings[ni])).view(1,batch_size,decoder.input_size)
        decoder_input_variable = decoder_input_variable.cuda() if use_cuda else decoder_input_variable
        
        idx = [word2index.get(sentence[i+1],0) for sentence in raw_sentences]
        
        target = Variable(torch.LongTensor(idx)).cuda() if use_cuda else Variable(torch.LongTensor(idx))
        
        prediction = decoder_output.view(batch_size,vocabularySize)
        
        mask = torch.ones(batch_size)
        for j in range(batch_size):
            if i >= lengths[j]:
                mask[j] = 0

        mask = Variable(mask)
        loss += torch.mul(criterion(prediction,target),mask)
        
        for k in range(len(ni)):
            if ni[k][0] == EOS_idx:
                lengths[k] = -1
                
    loss = torch.sum(loss)/batch_size
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0]/max_length

In [ ]:
learning_rate = 3*10**(-4)

hidden_size = 300
input_size = vocabularySize
output_size = vocabularySize

encoder = EncoderLSTM(input_size,hidden_size).cuda() if use_cuda else EncoderLSTM(input_size,hidden_size)
decoder = DecoderLSTM(input_size,hidden_size,output_size).cuda() if use_cuda else DecoderLSTM(input_size,hidden_size,output_size)

encoder_optimizer = torch.optim.Adam(encoder.parameters(),lr = learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss(reduce= False)

batch_size = 5
test_input = train_sentences[0:5]
train_enc_dec_batch(batch_size,test_input, encoder,decoder,encoder_optimizer, decoder_optimizer, criterion)

In [1]:
# Train the model and monitor the loss. Remember to use Adam optimizer and CrossEntropyLoss
plot_losses = []
print_loss_total = 0  # Reset every print_every
plot_loss_total = 0

def trainIters(encoder,decoder, n_iters,batch_size, print_every=1000, plot_every=100, learning_rate=0.01):
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    encoder_optimizer = torch.optim.Adam(encoder.parameters(),lr = learning_rate)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(),lr = learning_rate)
    criterion = nn.CrossEntropyLoss(reduce = False)
    
    n_batches = n_iters//batch_size
    
    for iter in range(1,n_batches - 1 ):
        
        target_variable = train_sentences[(iter-1)*batch_size:(iter)*batch_size]
        #target_variable = [train_sentences[0]]*5
        
        loss = train_enc_dec_batch(batch_size,target_variable, encoder,decoder,
                             encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            
            print_loss_avg = print_loss_total/print_every
            print_loss_total = 0
            
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_batches),
                                         iter, iter / n_batches * 100, print_loss_avg))
            
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            

In [361]:
learning_rate = 5*10**(-4)

hidden_size = 300
input_size = vocabularySize
output_size = vocabularySize

encoder = EncoderLSTM(input_size,hidden_size).cuda() if use_cuda else EncoderLSTM(input_size,hidden_size)
decoder = DecoderLSTM(input_size,hidden_size,output_size).cuda() if use_cuda else DecoderLSTM(input_size,hidden_size,output_size)

trainIters(encoder,decoder, 415000, 50 , 2 , 100, learning_rate)            

0m 0s (- 66m 8s) (2 0%) 3.9698
0m 1s (- 59m 41s) (4 0%) 4.8023
0m 2s (- 58m 23s) (6 0%) 4.7724
0m 3s (- 57m 15s) (8 0%) 4.7048
0m 4s (- 57m 22s) (10 0%) 4.8124
0m 4s (- 57m 29s) (12 0%) 4.2676
0m 5s (- 57m 22s) (14 0%) 4.5262
0m 6s (- 57m 44s) (16 0%) 3.8523
0m 7s (- 58m 3s) (18 0%) 3.6620
0m 8s (- 59m 33s) (20 0%) 3.3376
0m 9s (- 60m 32s) (22 0%) 3.0097
0m 10s (- 62m 11s) (24 0%) 2.8401
0m 12s (- 63m 53s) (26 0%) 2.5686
0m 13s (- 65m 24s) (28 0%) 2.7899
0m 14s (- 65m 29s) (30 0%) 2.2609
0m 15s (- 65m 17s) (32 0%) 2.6459
0m 16s (- 67m 8s) (34 0%) 2.5346
0m 17s (- 68m 4s) (36 0%) 2.6641
0m 18s (- 67m 49s) (38 0%) 2.6897
0m 20s (- 70m 5s) (40 0%) 2.4284
0m 22s (- 72m 45s) (42 0%) 2.0436
0m 23s (- 74m 8s) (44 0%) 2.4061
0m 25s (- 74m 56s) (46 0%) 2.2779
0m 25s (- 74m 7s) (48 0%) 2.6760
0m 26s (- 73m 36s) (50 0%) 2.3283
0m 27s (- 72m 56s) (52 0%) 2.7165
0m 29s (- 74m 27s) (54 0%) 2.1686
0m 30s (- 74m 48s) (56 0%) 2.0093
0m 31s (- 74m 17s) (58 0%) 2.3419
0m 32s (- 73m 49s) (60 0%) 2.4192
0m

4m 4s (- 66m 16s) (480 5%) 1.8346
4m 4s (- 66m 12s) (482 5%) 1.7166
4m 5s (- 66m 11s) (484 5%) 1.5922
4m 7s (- 66m 12s) (486 5%) 2.1286
4m 7s (- 66m 8s) (488 5%) 2.0847
4m 8s (- 66m 8s) (490 5%) 1.7295
4m 9s (- 66m 6s) (492 5%) 2.2520
4m 11s (- 66m 18s) (494 5%) 1.2014
4m 13s (- 66m 22s) (496 5%) 1.8140
4m 14s (- 66m 24s) (498 6%) 2.2863
4m 15s (- 66m 20s) (500 6%) 2.1727
4m 16s (- 66m 26s) (502 6%) 1.5935
4m 17s (- 66m 22s) (504 6%) 2.2031
4m 18s (- 66m 21s) (506 6%) 2.1482
4m 19s (- 66m 20s) (508 6%) 2.0468
4m 20s (- 66m 21s) (510 6%) 1.9313
4m 21s (- 66m 19s) (512 6%) 1.7109
4m 22s (- 66m 16s) (514 6%) 1.8084
4m 24s (- 66m 22s) (516 6%) 1.1158
4m 24s (- 66m 18s) (518 6%) 1.9341
4m 25s (- 66m 14s) (520 6%) 2.1726
4m 26s (- 66m 11s) (522 6%) 1.8382
4m 27s (- 66m 8s) (524 6%) 2.2148
4m 28s (- 66m 6s) (526 6%) 1.8741
4m 29s (- 66m 1s) (528 6%) 1.9509
4m 30s (- 65m 58s) (530 6%) 1.9053
4m 30s (- 65m 55s) (532 6%) 1.8930
4m 31s (- 65m 51s) (534 6%) 1.9620
4m 32s (- 65m 48s) (536 6%) 1.667

7m 52s (- 60m 54s) (950 11%) 2.0813
7m 53s (- 60m 52s) (952 11%) 2.2639
7m 54s (- 60m 50s) (954 11%) 2.2052
7m 55s (- 60m 49s) (956 11%) 1.8844
7m 55s (- 60m 47s) (958 11%) 2.3765
7m 56s (- 60m 45s) (960 11%) 2.2256
7m 57s (- 60m 42s) (962 11%) 2.4359
7m 58s (- 60m 41s) (964 11%) 2.3893
7m 59s (- 60m 40s) (966 11%) 2.0198
8m 0s (- 60m 40s) (968 11%) 2.1424
8m 1s (- 60m 42s) (970 11%) 2.2406
8m 3s (- 60m 43s) (972 11%) 2.1696
8m 4s (- 60m 45s) (974 11%) 2.1451
8m 6s (- 60m 47s) (976 11%) 1.8591
8m 7s (- 60m 49s) (978 11%) 1.5913
8m 8s (- 60m 51s) (980 11%) 2.0168
8m 10s (- 60m 53s) (982 11%) 2.0582
8m 11s (- 60m 51s) (984 11%) 2.1051
8m 12s (- 60m 51s) (986 11%) 2.2299
8m 13s (- 60m 52s) (988 11%) 1.7627
8m 14s (- 60m 52s) (990 11%) 2.0017
8m 16s (- 60m 54s) (992 11%) 1.8801
8m 17s (- 60m 55s) (994 11%) 2.0870
8m 18s (- 60m 52s) (996 12%) 2.5159
8m 19s (- 60m 52s) (998 12%) 2.4807
8m 20s (- 60m 54s) (1000 12%) 2.2319
8m 21s (- 60m 52s) (1002 12%) 2.2303
8m 22s (- 60m 49s) (1004 12%) 2.2

11m 47s (- 58m 24s) (1394 16%) 2.2002
11m 48s (- 58m 23s) (1396 16%) 2.0525
11m 49s (- 58m 22s) (1398 16%) 1.7840
11m 50s (- 58m 21s) (1400 16%) 1.8065
11m 51s (- 58m 20s) (1402 16%) 1.9253
11m 52s (- 58m 17s) (1404 16%) 2.4556
11m 53s (- 58m 18s) (1406 16%) 1.4295
11m 54s (- 58m 17s) (1408 16%) 2.1144
11m 55s (- 58m 16s) (1410 16%) 1.7862
11m 56s (- 58m 14s) (1412 17%) 2.0769
11m 57s (- 58m 13s) (1414 17%) 1.9046
11m 58s (- 58m 12s) (1416 17%) 2.1776
11m 59s (- 58m 11s) (1418 17%) 1.7117
12m 0s (- 58m 9s) (1420 17%) 2.3275
12m 1s (- 58m 9s) (1422 17%) 1.7569
12m 2s (- 58m 9s) (1424 17%) 1.5076
12m 3s (- 58m 8s) (1426 17%) 2.1864
12m 4s (- 58m 6s) (1428 17%) 2.3061
12m 5s (- 58m 6s) (1430 17%) 1.5502
12m 6s (- 58m 4s) (1432 17%) 2.3754
12m 7s (- 58m 3s) (1434 17%) 2.2536
12m 8s (- 58m 1s) (1436 17%) 1.8779
12m 9s (- 58m 0s) (1438 17%) 1.9791
12m 10s (- 58m 1s) (1440 17%) 1.9135
12m 11s (- 58m 0s) (1442 17%) 1.9974
12m 12s (- 57m 58s) (1444 17%) 2.1251
12m 13s (- 57m 57s) (1446 17%) 2.1

15m 26s (- 54m 34s) (1830 22%) 1.5367
15m 27s (- 54m 33s) (1832 22%) 1.5207
15m 27s (- 54m 31s) (1834 22%) 2.1585
15m 28s (- 54m 29s) (1836 22%) 2.0710
15m 29s (- 54m 28s) (1838 22%) 2.0562
15m 30s (- 54m 27s) (1840 22%) 1.6706
15m 31s (- 54m 25s) (1842 22%) 2.1915
15m 32s (- 54m 25s) (1844 22%) 1.4829
15m 33s (- 54m 24s) (1846 22%) 1.7509
15m 34s (- 54m 22s) (1848 22%) 1.8368
15m 35s (- 54m 21s) (1850 22%) 2.0108
15m 36s (- 54m 20s) (1852 22%) 1.8103
15m 37s (- 54m 19s) (1854 22%) 1.9386
15m 38s (- 54m 18s) (1856 22%) 2.0468
15m 40s (- 54m 19s) (1858 22%) 1.7099
15m 41s (- 54m 19s) (1860 22%) 1.8980
15m 42s (- 54m 19s) (1862 22%) 1.7386
15m 43s (- 54m 19s) (1864 22%) 2.2741
15m 44s (- 54m 17s) (1866 22%) 2.3361
15m 45s (- 54m 15s) (1868 22%) 1.9015
15m 46s (- 54m 14s) (1870 22%) 1.4958
15m 47s (- 54m 14s) (1872 22%) 1.5401
15m 49s (- 54m 15s) (1874 22%) 0.8495
15m 50s (- 54m 13s) (1876 22%) 2.1467
15m 50s (- 54m 11s) (1878 22%) 1.8369
15m 52s (- 54m 12s) (1880 22%) 1.5676
15m 53s (- 5

...

51m 7s (- 17m 25s) (6190 74%) 0.7007
51m 8s (- 17m 24s) (6192 74%) 0.7468
51m 9s (- 17m 23s) (6194 74%) 0.6367
51m 10s (- 17m 22s) (6196 74%) 0.4626
51m 11s (- 17m 21s) (6198 74%) 0.7132
51m 12s (- 17m 20s) (6200 74%) 0.6471
51m 13s (- 17m 19s) (6202 74%) 0.6254
51m 14s (- 17m 18s) (6204 74%) 0.5685
51m 14s (- 17m 17s) (6206 74%) 0.5900
51m 15s (- 17m 16s) (6208 74%) 0.5380
51m 16s (- 17m 15s) (6210 74%) 0.7901
51m 17s (- 17m 14s) (6212 74%) 0.7592
51m 18s (- 17m 13s) (6214 74%) 0.5995
51m 19s (- 17m 12s) (6216 74%) 0.7552
51m 19s (- 17m 11s) (6218 74%) 0.6887
51m 20s (- 17m 10s) (6220 74%) 0.6277
51m 21s (- 17m 9s) (6222 74%) 0.6417
51m 22s (- 17m 8s) (6224 74%) 0.6527
51m 23s (- 17m 7s) (6226 75%) 0.7260
51m 23s (- 17m 5s) (6228 75%) 0.6555
51m 24s (- 17m 4s) (6230 75%) 0.7147
51m 25s (- 17m 3s) (6232 75%) 0.6209
51m 26s (- 17m 2s) (6234 75%) 0.6756
51m 27s (- 17m 1s) (6236 75%) 0.5667
51m 28s (- 17m 0s) (6238 75%) 0.7302
51m 29s (- 16m 59s) (6240 75%) 0.4953
51m 30s (- 16m 58s) (624

54m 25s (- 13m 44s) (6626 79%) 0.9060
54m 26s (- 13m 43s) (6628 79%) 0.5949
54m 27s (- 13m 42s) (6630 79%) 0.7934
54m 28s (- 13m 41s) (6632 79%) 0.7201
54m 28s (- 13m 40s) (6634 79%) 0.7687
54m 29s (- 13m 39s) (6636 79%) 0.7067
54m 31s (- 13m 38s) (6638 79%) 0.7092
54m 32s (- 13m 38s) (6640 80%) 0.6879
54m 33s (- 13m 37s) (6642 80%) 0.7942
54m 34s (- 13m 36s) (6644 80%) 0.5487
54m 35s (- 13m 35s) (6646 80%) 0.6144
54m 37s (- 13m 34s) (6648 80%) 0.7570
54m 38s (- 13m 33s) (6650 80%) 0.6682
54m 39s (- 13m 32s) (6652 80%) 0.7125
54m 40s (- 13m 31s) (6654 80%) 0.5057
54m 41s (- 13m 30s) (6656 80%) 0.5246
54m 42s (- 13m 29s) (6658 80%) 0.5145
54m 43s (- 13m 28s) (6660 80%) 0.6449
54m 44s (- 13m 27s) (6662 80%) 0.7503
54m 45s (- 13m 26s) (6664 80%) 0.7102
54m 46s (- 13m 25s) (6666 80%) 0.6296
54m 47s (- 13m 24s) (6668 80%) 0.5693
54m 48s (- 13m 23s) (6670 80%) 0.5543
54m 49s (- 13m 22s) (6672 80%) 0.4149
54m 50s (- 13m 21s) (6674 80%) 0.6993
54m 51s (- 13m 20s) (6676 80%) 0.6514
54m 52s (- 1

58m 4s (- 10m 10s) (7062 85%) 0.3657
58m 5s (- 10m 9s) (7064 85%) 0.5795
58m 6s (- 10m 8s) (7066 85%) 0.4512
58m 7s (- 10m 7s) (7068 85%) 0.5180
58m 8s (- 10m 6s) (7070 85%) 0.4456
58m 8s (- 10m 5s) (7072 85%) 0.4156
58m 9s (- 10m 4s) (7074 85%) 0.3832
58m 10s (- 10m 3s) (7076 85%) 0.5207
58m 11s (- 10m 2s) (7078 85%) 0.5596
58m 12s (- 10m 1s) (7080 85%) 0.5328
58m 13s (- 10m 0s) (7082 85%) 0.6222
58m 14s (- 9m 59s) (7084 85%) 0.7034
58m 14s (- 9m 58s) (7086 85%) 0.5486
58m 15s (- 9m 57s) (7088 85%) 0.4917
58m 17s (- 9m 56s) (7090 85%) 0.4667
58m 18s (- 9m 55s) (7092 85%) 0.5363
58m 19s (- 9m 54s) (7094 85%) 0.5475
58m 20s (- 9m 53s) (7096 85%) 0.4898
58m 21s (- 9m 52s) (7098 85%) 0.4289
58m 22s (- 9m 51s) (7100 85%) 0.5254
58m 23s (- 9m 50s) (7102 85%) 0.5603
58m 23s (- 9m 49s) (7104 85%) 0.6691
58m 25s (- 9m 48s) (7106 85%) 0.5435
58m 26s (- 9m 47s) (7108 85%) 0.4256
58m 27s (- 9m 47s) (7110 85%) 0.5931
58m 28s (- 9m 46s) (7112 85%) 0.5328
58m 29s (- 9m 45s) (7114 85%) 0.4930
58m 30s

61m 46s (- 6m 29s) (7510 90%) 0.4888
61m 46s (- 6m 28s) (7512 90%) 0.4901
61m 47s (- 6m 27s) (7514 90%) 0.6016
61m 48s (- 6m 26s) (7516 90%) 0.4539
61m 49s (- 6m 25s) (7518 90%) 0.5522
61m 50s (- 6m 24s) (7520 90%) 0.5805
61m 50s (- 6m 23s) (7522 90%) 0.5462
61m 51s (- 6m 22s) (7524 90%) 0.4103
61m 52s (- 6m 21s) (7526 90%) 0.4990
61m 53s (- 6m 20s) (7528 90%) 0.6360
61m 54s (- 6m 19s) (7530 90%) 0.6182
61m 54s (- 6m 18s) (7532 90%) 0.4870
61m 55s (- 6m 17s) (7534 90%) 0.5389
61m 56s (- 6m 16s) (7536 90%) 0.4901
61m 57s (- 6m 15s) (7538 90%) 0.5436
61m 58s (- 6m 14s) (7540 90%) 0.4774
61m 59s (- 6m 13s) (7542 90%) 0.5137
61m 59s (- 6m 12s) (7544 90%) 0.7089
62m 0s (- 6m 11s) (7546 90%) 0.4979
62m 1s (- 6m 10s) (7548 90%) 0.4712
62m 2s (- 6m 9s) (7550 90%) 0.4843
62m 3s (- 6m 8s) (7552 90%) 0.4446
62m 3s (- 6m 7s) (7554 91%) 0.4681
62m 4s (- 6m 6s) (7556 91%) 0.6434
62m 5s (- 6m 5s) (7558 91%) 0.5038
62m 6s (- 6m 4s) (7560 91%) 0.4267
62m 7s (- 6m 3s) (7562 91%) 0.5005
62m 8s (- 6m 2s) 

65m 4s (- 2m 47s) (7958 95%) 0.4787
65m 4s (- 2m 46s) (7960 95%) 0.6493
65m 5s (- 2m 45s) (7962 95%) 0.6023
65m 6s (- 2m 44s) (7964 95%) 0.5653
65m 7s (- 2m 43s) (7966 95%) 0.4895
65m 8s (- 2m 42s) (7968 96%) 0.6281
65m 8s (- 2m 41s) (7970 96%) 0.4997
65m 9s (- 2m 40s) (7972 96%) 0.5009
65m 10s (- 2m 39s) (7974 96%) 0.5622
65m 11s (- 2m 38s) (7976 96%) 0.5729
65m 12s (- 2m 37s) (7978 96%) 0.4726
65m 13s (- 2m 36s) (7980 96%) 0.4305
65m 14s (- 2m 35s) (7982 96%) 0.5178
65m 15s (- 2m 34s) (7984 96%) 0.4910
65m 16s (- 2m 33s) (7986 96%) 0.4375
65m 16s (- 2m 32s) (7988 96%) 0.6663
65m 17s (- 2m 32s) (7990 96%) 0.5123
65m 18s (- 2m 31s) (7992 96%) 0.5982
65m 19s (- 2m 30s) (7994 96%) 0.4342
65m 20s (- 2m 29s) (7996 96%) 0.4216
65m 21s (- 2m 28s) (7998 96%) 0.5342
65m 22s (- 2m 27s) (8000 96%) 0.6194
65m 23s (- 2m 26s) (8002 96%) 0.4205
65m 23s (- 2m 25s) (8004 96%) 0.5740
65m 24s (- 2m 24s) (8006 96%) 0.4628
65m 25s (- 2m 23s) (8008 96%) 0.4163
65m 26s (- 2m 22s) (8010 96%) 0.5201
65m 27s (

In [ ]:
torch.save(encoder,'encoder_batch_full.pt')
torch.save(decoder,'decoder_batch_full.pt')

In [10]:
encoder = torch.load('encoder_batch_full.pt')
decoder = torch.load('decoder_batch_full.pt')

In [26]:
#test result for training repeatedly on the same sentence

test_sentence = val_sentences[2]
print(test_sentence)
temp = seq2seq_inference(test_sentence, embeddings=one_hot_embeddings, max_length=maxSequenceLength)
print(temp)
test_sentence = val_sentences[9]
print(test_sentence)
temp = seq2seq_inference(test_sentence, embeddings=one_hot_embeddings, max_length=maxSequenceLength)
print(temp)

Four horses are skattered around a small water hole.
['four', 'adults', '<UNK>', '<UNK>', 'around', 'a', 'large', 'group', 'water', '.', '<EOS>']
A baseball player standing next to home plate with a bat.
['a', 'baseball', 'player', 'standing', 'next', 'to', 'home', 'plate', 'with', 'a', 'bat', '.', '<EOS>']


In [362]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('objs.pkl', 'wb') as f:  
    pickle.dump([word2index, vocabulary, w2v_embeddings], f)

In [11]:
import pickle
# Getting back the objects:
with open('objs.pkl','rb') as f:  # Python 3: open(..., 'rb')
    word2index,vocabulary,w2v_embeddings = pickle.load(f)

In [30]:
#Computing bleu score for one hot-encoding with batch
score = 0
for i in range(500):
    test_sentence = val_sentences[i]
    #print(test_sentence)
    temp = seq2seq_inference(test_sentence, embeddings=one_hot_embeddings, max_length=maxSequenceLength)
    score += compute_bleu(test_sentence,temp[1:-1])
    
print(score/500)

/home/suhail/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/home/suhail/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/home/suhail/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6653662992590728
